# Basics about quantum simulation

In [1]:
using Yao, Yao.Blocks
using LinearAlgebra
using Test, BenchmarkTools

┌ Info: Recompiling stale cache file /home/leo/.julia/compiled/v1.0/Yao/Hg5qP.ji for Yao [5a1af4f6-c801-11e8-08ea-1bad16a356b2]
└ @ Base loading.jl:1184


In [2]:
reg = zero_state(10)

DefaultRegister{1, Complex{Float64}}
    active qubits: 10/10

In [3]:
focus!(reg, 1:5)

DefaultRegister{1, Complex{Float64}}
    active qubits: 5/10

In [4]:
nactive(reg)

5

In [5]:
relax!(reg)

DefaultRegister{1, Complex{Float64}}
    active qubits: 10/10

In [6]:
focus!(reg, [3,1,5]) do r
    r.state = randn(8, 8) * r.state
    r
end

DefaultRegister{1, Complex{Float64}}
    active qubits: 10/10

In [7]:
measure(reg, 10)

10-element Array{Int64,1}:
  0
 21
 21
 20
 21
  0
 20
 20
 20
 20

In [44]:
[measure!(reg) for i=1:10]

10-element Array{Array{Int64,1},1}:
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]

In [45]:
[measure_reset!(reg, val=0b0110011100) for i=1:10]

10-element Array{Array{Int64,1},1}:
 [5]  
 [412]
 [412]
 [412]
 [412]
 [412]
 [412]
 [412]
 [412]
 [412]

In [46]:
measure_remove!(reg |> focus!(3,4,2,1))

1-element Array{Int64,1}:
 3

In [49]:
reg

DefaultRegister{1, Complex{Float64}}
    active qubits: 6/6

In [51]:
reg3 = relax!(reg) ⊗ rand_state(4)

DefaultRegister{1, Complex{Float64}}
    active qubits: 10/10

In [56]:
select(reg3, 0b0011100010) |> statevec

1-element Array{Complex{Float64},1}:
 0.0 + 0.0im

In [8]:
ψ1 = rand_state(6, 3)
ψ2 = rand_state(6, 3)
fidelity(ψ1, ψ2)

3-element Array{Float64,1}:
 0.12627187929637737
 0.10963506150124552
 0.07092350044992166

In [74]:
import Base: *, iterate, broadcastable
*(bra::ConjRegister{1}, ket::DefaultRegister{1}) = statevec(bra) * statevec(ket)
*(bra::ConjRegister, ket::DefaultRegister) = bra .* ket
broadcastable(reg::AbstractRegister{1}) = Ref{reg}
iterate(reg::AbstractRegister{1}, state=1) = state == 1 ? (reg, 2) : nothing

function broadcastable(reg::DefaultRegister{B}) where B
    st = reg |> rank3
    Tuple(register(view(st, :, :, i)) for i = 1:B)
end

function iterate(reg::DefaultRegister{B}, state=1) where B
    state > B && return nothing
    st = reg |> rank3
    register(view(st, :, :, state)), state+1
end

function broadcastable(reg::ConjRegister{B}) where B
    st = reg |> rank3
    Tuple(register(view(st, :, :, i))' for i = 1:B)
end

function iterate(reg::ConjRegister{B}, state=1) where B
    state > B && return nothing
    st = reg |> rank3
    register(view(st, :, :, state))', state+1
end

iterate (generic function with 273 methods)

In [9]:
focus!(ψ1, [1,2,4])' * focus!(ψ2, [1,2,4])

MethodError: MethodError: no method matching rank3(::Adjoint{Complex{Float64},DefaultRegister{3,Complex{Float64},Array{Complex{Float64},2}}})
Closest candidates are:
  rank3(!Matched::DefaultRegister{B,T,MT} where MT<:AbstractArray{T,2} where T) where B at /home/leo/.julia/dev/Yao/src/Registers/Default.jl:46

# Batched Registers

In [12]:
reg = rand_state(10, 8)

DefaultRegister{8, Complex{Float64}}
    active qubits: 10/10

In [16]:
@. reg * 5 - 4 * reg == (reg...,)

MethodError: MethodError: no method matching iterate(::DefaultRegister{8,Complex{Float64},Array{Complex{Float64},2}})
Closest candidates are:
  iterate(!Matched::Core.SimpleVector) at essentials.jl:578
  iterate(!Matched::Core.SimpleVector, !Matched::Any) at essentials.jl:578
  iterate(!Matched::ExponentialBackOff) at error.jl:171
  ...

# Blocks

In [17]:
X

X gate

In [19]:
mat(X)

2×2 LuxurySparse.PermMatrix{Complex{Float64},Int64,Array{Complex{Float64},1},Array{Int64,1}}:
    0       1.0+0.0im
 1.0+0.0im     0     

In [20]:
px = put(3, 1=>X)

Total: 3, DataType: Complex{Float64}
put on (1)
└─ X gate


In [21]:
mat(px)

8×8 LuxurySparse.PermMatrix{Complex{Float64},Int64,Array{Complex{Float64},1},Array{Int64,1}}:
    0       1.0+0.0im     0       …     0          0          0     
 1.0+0.0im     0          0             0          0          0     
    0          0          0             0          0          0     
    0          0       1.0+0.0im        0          0          0     
    0          0          0          1.0+0.0im     0          0     
    0          0          0       …     0          0          0     
    0          0          0             0          0       1.0+0.0im
    0          0          0             0       1.0+0.0im     0     

In [24]:
2X

Total: 1, DataType: Complex{Float64}
[2] X gate


In [25]:
mat(2X)

2×2 LuxurySparse.PermMatrix{Complex{Float64},Int64,Array{Complex{Float64},1},Array{Int64,1}}:
    0       2.0+0.0im
 2.0+0.0im     0     

In [27]:
cx = control(3, 3, 1=>X)

Total: 3, DataType: Complex{Float64}
control(3)
└─ (1,)=>X gate


In [28]:
mat(cx)

8×8 LuxurySparse.PermMatrix{Complex{Float64},Int64,Array{Complex{Float64},1},Array{Int64,1}}:
 1.0+0.0im     0          0       …     0          0          0     
    0       1.0+0.0im     0             0          0          0     
    0          0       1.0+0.0im        0          0          0     
    0          0          0             0          0          0     
    0          0          0          1.0+0.0im     0          0     
    0          0          0       …     0          0          0     
    0          0          0             0          0       1.0+0.0im
    0          0          0             0       1.0+0.0im     0     

In [28]:
struct B end

In [29]:
Base.length(x::B) = 5
Base.getindex(a::B, i) = 3

In [30]:
B().^2

MethodError: MethodError: no method matching iterate(::B)
Closest candidates are:
  iterate(!Matched::Core.SimpleVector) at essentials.jl:578
  iterate(!Matched::Core.SimpleVector, !Matched::Any) at essentials.jl:578
  iterate(!Matched::ExponentialBackOff) at error.jl:171
  ...